In [11]:
from pathlib import Path
import pandas as pd
import rasterio
import numpy as np

BASE = Path("../VerimGören/notebooks/hwsd_data")  # klasörün tam yolunu ver
RASTER = BASE/"hwsd.bil"
MDB    = BASE/"HWSD.mdb"


In [13]:
def mu_global_from_latlon(lat: float, lon: float, raster_path=RASTER) -> int:
    with rasterio.open(raster_path) as src:
        row, col = src.index(lon, lat)         # (x=lon, y=lat)
        mu = src.read(1)[row, col]
    mu = int(mu)
    if mu <= 0:
        raise ValueError(f"Geçersiz MU_GLOBAL (mu={mu}) — nokta su/şehir/buz olabilir.")
    return mu


In [23]:
from pathlib import Path

BASE = Path.cwd() / "hwsd_data"   # veya Path("./hwsd_data")
RASTER = BASE / "hwsd.bil"
MDB    = BASE / "HWSD.mdb"

print("BASE:", BASE)
print("MDB exists?", MDB.exists())
print("RASTER exists?", RASTER.exists())


BASE: C:\Users\ataka\Desktop\VerimGören\notebooks\hwsd_data
MDB exists? True
RASTER exists? True


In [25]:
import pyodbc, pandas as pd
from pathlib import Path

def read_table_access(table, mdb_path=MDB):
    mdb_path = Path(mdb_path).resolve()
    if not mdb_path.exists():
        raise FileNotFoundError(mdb_path)
    conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={str(mdb_path).replace('\\','/')};"
    cn = pyodbc.connect(conn_str)
    try:
        return pd.read_sql(f"SELECT * FROM {table}", cn)
    finally:
        cn.close()

# deneme: tablo isimlerini sırayla okuyalım
hwsd = read_table_access("HWSD_DATA")      # ana tablo
tex  = read_table_access("D_TEXTURE")      # sözlük
awc  = read_table_access("D_AWC")
phase= read_table_access("D_PHASE")

print("HWSD_DATA satır sayısı:", len(hwsd))
print("D_TEXTURE satır sayısı:", len(tex))
print("D_AWC satır sayısı:", len(awc))
print("D_PHASE satır sayısı:", len(phase))


C:\Users\ataka\AppData\Local\Temp\ipykernel_2240\1651761567.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(f"SELECT * FROM {table}", cn)


HWSD_DATA satır sayısı: 48148
D_TEXTURE satır sayısı: 4
D_AWC satır sayısı: 7
D_PHASE satır sayısı: 31


In [33]:
# ——— 0) Yardımcı: sütunları normalize et (trim + UPPERCASE) ———
import pandas as pd

def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Sütun adlarını kırpıp BÜYÜK HARF yapar; adlandırma farklarını tolere eder."""
    df = df.copy()
    df.columns = [c.strip().upper() for c in df.columns]
    return df

# ——— 1) Tabloları normalize et ———
hwsd  = normalize_cols(hwsd)     # HWSD_DATA
tex   = normalize_cols(tex)      # D_TEXTURE  (sende: CODE, VALUE)
awc   = normalize_cols(awc)      # D_AWC      (sende: CODE, VALUE)
phase = normalize_cols(phase)    # D_PHASE    (sende: CODE, VALUE)

print("HWSD_DATA cols (örnek):", list(hwsd.columns)[:20])
print("D_TEXTURE cols:", list(tex.columns))
print("D_AWC cols:", list(awc.columns))
print("D_PHASE cols:", list(phase.columns))

# ——— 2) Lookup tablolarında CODE sabit; açıklama 'VALUE' ———
#     (Bazı dağıtımlarda 'DESCRIPTION' olur; biz ikisine de bakacak şekilde yazıyoruz.)
def get_code_desc_cols(df):
    cols = set(df.columns)
    code = "CODE" if "CODE" in cols else None
    # açıklama için önce DESCRIPTION'a, yoksa VALUE'a bak
    desc = "DESCRIPTION" if "DESCRIPTION" in cols else ("VALUE" if "VALUE" in cols else None)
    return code, desc

tex_code, tex_desc = get_code_desc_cols(tex)
awc_code, awc_desc = get_code_desc_cols(awc)
ph_code,  ph_desc  = get_code_desc_cols(phase)

assert tex_code and tex_desc,  "D_TEXTURE'de CODE/desc (DESCRIPTION/ VALUE) bulunamadı."
assert awc_code and awc_desc,  "D_AWC'da CODE/desc (DESCRIPTION/ VALUE) bulunamadı."
assert ph_code  and ph_desc,   "D_PHASE'de CODE/desc (DESCRIPTION/ VALUE) bulunamadı."

# ——— 3) Lookup tablolarını daralt ve isimlendir ———
tex_lut = tex.rename(columns={tex_code:"T_TEXTURE", tex_desc:"T_TEXTURE_DESC"})[["T_TEXTURE","T_TEXTURE_DESC"]]
awc_lut = awc.rename(columns={awc_code:"AWC_CLASS", awc_desc:"AWC_DESC"})[["AWC_CLASS","AWC_DESC"]]
ph_lut  = phase.rename(columns={ph_code:"PHASE_CODE", ph_desc:"PHASE_DESC"})[["PHASE_CODE","PHASE_DESC"]]

# anahtarların numerik olmasını sağla (eşleşme sorunlarını önler)
for c in ["T_TEXTURE","AWC_CLASS","PHASE1","PHASE2"]:
    if c in hwsd.columns:
        hwsd[c] = pd.to_numeric(hwsd[c], errors="coerce")
for c in ["T_TEXTURE"]:
    if c in tex_lut.columns:
        tex_lut[c] = pd.to_numeric(tex_lut[c], errors="coerce")
for c in ["AWC_CLASS"]:
    if c in awc_lut.columns:
        awc_lut[c] = pd.to_numeric(awc_lut[c], errors="coerce")
for c in ["PHASE_CODE"]:
    if c in ph_lut.columns:
        ph_lut[c] = pd.to_numeric(ph_lut[c], errors="coerce")

# ——— 4) Güvenli JOIN ———
df = hwsd.copy()

if "T_TEXTURE" in df.columns:
    df = df.merge(tex_lut, on="T_TEXTURE", how="left")
else:
    print("Uyarı: HWSD_DATA içinde T_TEXTURE yok; dokusal açıklama eklenmeyecek.")

if "AWC_CLASS" in df.columns:
    df = df.merge(awc_lut, on="AWC_CLASS", how="left")
else:
    print("Uyarı: HWSD_DATA içinde AWC_CLASS yok; AWC açıklaması eklenmeyecek.")

# PHASE1 ve PHASE2 için aynı sözlüğü iki kez kullanıyoruz
if "PHASE1" in df.columns:
    df = (df.merge(ph_lut, left_on="PHASE1", right_on="PHASE_CODE", how="left")
            .rename(columns={"PHASE_DESC":"PHASE1_DESC"})
            .drop(columns=["PHASE_CODE"]))
if "PHASE2" in df.columns:
    df = (df.merge(ph_lut, left_on="PHASE2", right_on="PHASE_CODE", how="left")
            .rename(columns={"PHASE_DESC":"PHASE2_DESC"})
            .drop(columns=["PHASE_CODE"]))

print("JOIN sonrası şekil:", df.shape)


HWSD_DATA cols (örnek): ['ID', 'MU_GLOBAL', 'MU_SOURCE1', 'MU_SOURCE2', 'ISSOIL', 'SHARE', 'SEQ', 'SU_SYM74', 'SU_CODE74', 'SU_SYM85', 'SU_CODE85', 'SU_SYM90', 'SU_CODE90', 'T_TEXTURE', 'DRAINAGE', 'REF_DEPTH', 'AWC_CLASS', 'PHASE1', 'PHASE2', 'ROOTS']
D_TEXTURE cols: ['CODE', 'VALUE']
D_AWC cols: ['CODE', 'VALUE']
D_PHASE cols: ['CODE', 'VALUE']
JOIN sonrası şekil: (48148, 61)


In [35]:
# ——— 5) Her MU_GLOBAL için tek satır (dominant bileşen) ———
# SEQ (küçük → daha dominant), eşitlikte SHARE (büyük → tercih) kullanıyoruz.
dom = (df.sort_values(["MU_GLOBAL", 
                       "SEQ"   if "SEQ"   in df.columns else "MU_GLOBAL", 
                       "SHARE" if "SHARE" in df.columns else "MU_GLOBAL"],
                      ascending=[True, True, False])
         .groupby("MU_GLOBAL", as_index=False)
         .first())
print("Dominant tablo şekil:", dom.shape)


Dominant tablo şekil: (16327, 61)


In [37]:
# ——— 6) Nokta sorgu (lat,lon → MU_GLOBAL → tek satır bilgi) ———
import rasterio
from pathlib import Path

RASTER = Path(BASE)/"hwsd.bil"

def mu_global_from_latlon(lat, lon, raster_path=RASTER):
    """
    Raster (hwsd.bil) üzerinde (lat,lon) noktasındaki MU_GLOBAL'i döndürür.
    Not: rasterio.index(x=lon, y=lat)
    """
    with rasterio.open(raster_path) as src:
        r, c = src.index(lon, lat)
        mu = int(src.read(1)[r, c])
    if mu <= 0:
        raise ValueError(f"Geçersiz/non-soil MU_GLOBAL={mu} (göl/şehir/buz olabilir).")
    return mu

def hwsd_point_query(lat, lon, dom_df=dom):
    """
    Verilen konum için dominant HWSD satırını döndürür.
    - Rasterdan MU_GLOBAL okunur
    - Dominant tablo içinde arama yapılır
    - Sık kullanılan kolonların kısa bir alt kümesini geri verir
    """
    mu = mu_global_from_latlon(lat, lon)
    row = dom_df.loc[dom_df["MU_GLOBAL"] == mu]
    if row.empty:
        raise LookupError(f"MU_GLOBAL={mu} için kayıt bulunamadı.")
    cols = [
        "MU_GLOBAL","SEQ","SHARE",
        "T_TEXTURE","T_TEXTURE_DESC","AWC_CLASS","AWC_DESC",
        "T_CLAY","T_SILT","T_SAND","T_OC","T_PH_H2O","T_CEC_SOIL",
        "S_CLAY","S_OC","S_PH_H2O","S_CEC_SOIL",
        "PHASE1","PHASE1_DESC","PHASE2","PHASE2_DESC"
    ]
    cols = [c for c in cols if c in row.columns]
    return mu, row.iloc[0][cols]

# Hızlı test
mu, info = hwsd_point_query(36.0, 36.0)
print("MU_GLOBAL:", mu)
info


MU_GLOBAL: 3280


MU_GLOBAL         3280
SEQ                  1
SHARE             40.0
T_TEXTURE          3.0
T_TEXTURE_DESC    Fine
AWC_CLASS          2.0
AWC_DESC           125
T_CLAY            55.0
T_SILT            29.0
T_SAND            16.0
T_OC              0.75
T_PH_H2O           7.9
T_CEC_SOIL        44.0
S_CLAY            57.0
S_OC              0.45
S_PH_H2O           8.1
S_CEC_SOIL        43.0
PHASE1             NaN
PHASE1_DESC       None
PHASE2             NaN
PHASE2_DESC       None
Name: 1130, dtype: object

In [39]:
# ——— 7) (Opsiyonel) Kaydet: Parquet + CSV ———
(dom
 .to_parquet(BASE/"hwsd_units_dom.parquet", index=False))
dom.to_csv(BASE/"hwsd_units_dom.csv", index=False)
print("Kaydedildi:", BASE)


Kaydedildi: C:\Users\ataka\Desktop\VerimGören\notebooks\hwsd_data


In [56]:
# =========================================================
# HWSD — Tek Hücrelik Çözüm
# Koordinattan (lat,lon) → "insan-dili" rapor + tam satır
# =========================================================

from pathlib import Path
import pandas as pd
import rasterio
import pyodbc

# ---------------------------------------------------------
# 0) YOL AYARI —> KENDİ DİZİNİNE GÖRE DÜZENLE
#    Klasörde bu 4 dosya olmalı: HWSD.mdb, hwsd.bil, hwsd.hdr, hwsd.blw
# ---------------------------------------------------------
BASE   = Path(r"C:/Users/ataka/Desktop/VerimGören/notebooks/hwsd_data")
MDB    = BASE / "HWSD.mdb"
RASTER = BASE / "hwsd.bil"

# ---------------------------------------------------------
# 1) YARDIMCI FONKSİYONLAR
# ---------------------------------------------------------
def read_table_access(table_name, mdb_path=MDB) -> pd.DataFrame:
    """
    Access (.mdb) içinden 'table_name' tablosunu DataFrame olarak döndürür.
    - Yol mutlaklaştırılır ve forward slash'a çevrilir (Windows kaçış sorunlarını önler).
    - PyODBC ile bağlanır; pandas read_sql ile okur.
    """
    mdb_path = Path(mdb_path).resolve()
    if not mdb_path.exists():
        raise FileNotFoundError(f"HWSD.mdb bulunamadı: {mdb_path}")
    conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={str(mdb_path).replace('\\','/')};"
    cn = pyodbc.connect(conn_str)
    try:
        return pd.read_sql(f"SELECT * FROM {table_name}", cn)
    finally:
        cn.close()

def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    """
    Sütun adlarını normalize eder (strip + UPPERCASE).
    Adlandırma farklılıklarından doğan KeyError riskini azaltır.
    """
    out = df.copy()
    out.columns = [c.strip().upper() for c in df.columns]
    return out

def get_code_desc_cols(df: pd.DataFrame):
    """
    Lookup tablolarında 'CODE' ve açıklama kolonu tespit eder.
    Açıklama için önce 'DESCRIPTION'a bakar; yoksa 'VALUE' kabul eder.
    """
    cols = set(df.columns)
    code = "CODE" if "CODE" in cols else None
    desc = "DESCRIPTION" if "DESCRIPTION" in cols else ("VALUE" if "VALUE" in cols else None)
    return code, desc

def _num(x):
    """
    Güvenli sayısal dönüştürücü:
    - '3,5' gibi virgüllü değerleri destekler,
    - Dönüşemezse None döndürür.
    """
    if x is None:
        return None
    try:
        return float(str(x).replace(",", "."))
    except Exception:
        return None

def mu_global_from_latlon(lat: float, lon: float, raster_path=RASTER) -> int:
    """
    Raster (hwsd.bil) üzerinde (lat,lon) noktasındaki MU_GLOBAL değerini döndürür.
    Not: rasterio.index sırası (x=lon, y=lat)
    """
    rp = Path(raster_path).resolve()
    if not rp.exists():
        raise FileNotFoundError(f"hwsd.bil bulunamadı: {rp}")
    with rasterio.open(rp) as src:
        r, c = src.index(lon, lat)
        mu = int(src.read(1)[r, c])
    if mu <= 0:
        raise ValueError(f"Geçersiz/non-soil MU_GLOBAL={mu} (göl/şehir/buz olabilir).")
    return mu

# ---------------------------------------------------------
# 2) TABLOLARI OKU (HWSD_DATA + LOOKUP'LAR)
#    Bazı lookup tabloları bazı dağıtımlarda olmayabilir;
#    try/except ile okumaya çalışıp yoksa atlıyoruz.
# ---------------------------------------------------------
hwsd   = normalize_cols(read_table_access("HWSD_DATA"))

def safe_read_lookup(name):
    try:
        return normalize_cols(read_table_access(name))
    except Exception:
        return None

tex   = safe_read_lookup("D_TEXTURE")           # T_TEXTURE (coarse/medium/fine) — küçük sözlük
utex  = safe_read_lookup("D_USDA_TEX_CLASS")    # T_USDA_TEX_CLASS, S_USDA_TEX_CLASS → Clay/Loam...
awc   = safe_read_lookup("D_AWC")               # AWC_CLASS → mm/m
drn   = safe_read_lookup("D_DRAINAGE")          # DRAINAGE → açıklama
phase = safe_read_lookup("D_PHASE")             # PHASE kodları (opsiyonel)
sym90 = safe_read_lookup("D_SYMBOL90")          # SU_CODE90 → FAO-90 isim
sym   = safe_read_lookup("D_SYMBOL")            # SU_CODE → genel sembol (yedek)

# ---------------------------------------------------------
# 3) LOOKUP LUT'LARINI HAZIRLA (VARSA)
# ---------------------------------------------------------
tex_lut = None
if isinstance(tex, pd.DataFrame):
    t_code, t_desc = get_code_desc_cols(tex)
    if t_code and t_desc:
        tex_lut = tex.rename(columns={t_code:"T_TEXTURE", t_desc:"T_TEXTURE_DESC"})[["T_TEXTURE","T_TEXTURE_DESC"]]

usda_t_lut = usda_s_lut = None
if isinstance(utex, pd.DataFrame):
    u_code, u_desc = get_code_desc_cols(utex)
    if u_code and u_desc:
        usda_t_lut = utex.rename(columns={u_code:"T_USDA_TEX_CLASS", "DESCRIPTION":"T_USDA_TEX_DESC", "VALUE":"T_USDA_TEX_DESC"}) \
                         [["T_USDA_TEX_CLASS","T_USDA_TEX_DESC"]]
        usda_s_lut = utex.rename(columns={u_code:"S_USDA_TEX_CLASS", "DESCRIPTION":"S_USDA_TEX_DESC", "VALUE":"S_USDA_TEX_DESC"}) \
                         [["S_USDA_TEX_CLASS","S_USDA_TEX_DESC"]]

awc_lut = None
if isinstance(awc, pd.DataFrame):
    a_code, a_desc = get_code_desc_cols(awc)
    if a_code and a_desc:
        awc_lut = awc.rename(columns={a_code:"AWC_CLASS", a_desc:"AWC_MM_PER_M"})[["AWC_CLASS","AWC_MM_PER_M"]]

drn_lut = None
if isinstance(drn, pd.DataFrame):
    d_code, d_desc = get_code_desc_cols(drn)
    if d_code and d_desc:
        drn_lut = drn.rename(columns={d_code:"DRAINAGE", d_desc:"DRAINAGE_DESC"})[["DRAINAGE","DRAINAGE_DESC"]]

sym90_lut = None
if isinstance(sym90, pd.DataFrame):
    s_code, s_desc = get_code_desc_cols(sym90)
    if s_code and s_desc:
        sym90_lut = sym90.rename(columns={s_code:"SU_CODE90", s_desc:"FAO90_DESC"})[["SU_CODE90","FAO90_DESC"]]

# ---------------------------------------------------------
# 4) JOIN: HWSD_DATA + (VARSA) LOOKUP AÇIKLAMALARI
# ---------------------------------------------------------
df = hwsd.copy()

# T doku kaba/orta/ince (coarse/medium/fine)
if tex_lut is not None and "T_TEXTURE" in df.columns:
    df = df.merge(tex_lut, on="T_TEXTURE", how="left")

# USDA doku sınıfları (üst/alt) (örn. Clay / Loam / Sandy Loam)
if usda_t_lut is not None and "T_USDA_TEX_CLASS" in df.columns:
    df = df.merge(usda_t_lut, on="T_USDA_TEX_CLASS", how="left")
if usda_s_lut is not None and "S_USDA_TEX_CLASS" in df.columns:
    df = df.merge(usda_s_lut, on="S_USDA_TEX_CLASS", how="left")

# AWC sınıfı (mm/m)
if awc_lut is not None and "AWC_CLASS" in df.columns:
    df = df.merge(awc_lut, on="AWC_CLASS", how="left")

# Drenaj açıklaması
if drn_lut is not None and "DRAINAGE" in df.columns:
    df = df.merge(drn_lut, on="DRAINAGE", how="left")

# FAO-90 toprak grubu adı
if sym90_lut is not None and "SU_CODE90" in df.columns:
    df = df.merge(sym90_lut, on="SU_CODE90", how="left")

# ---------------------------------------------------------
# 5) JOIN SONRASI: SAYISAL OLMASI GEREKEN KOLONLARI NUMERİK YAP
#    (Bazı alanlar string gelebilir; kıyaslamalar için float'a çeviriyoruz.)
# ---------------------------------------------------------
num_cols = [
    "AWC_MM_PER_M",
    "T_PH_H2O","S_PH_H2O",
    "T_OC","S_OC",
    "T_CLAY","T_SILT","T_SAND",
    "S_CLAY","S_SILT","S_SAND",
    "T_ECE","S_ECE","T_ESP","S_ESP",
    "T_CEC_SOIL","S_CEC_SOIL","T_CEC_CLAY","S_CEC_CLAY",
    "T_BS","S_BS","T_TEB","S_TEB",
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", "."), errors="coerce")

# ---------------------------------------------------------
# 6) DOMİNANT BİLEŞEN: her MU_GLOBAL için tek satır
#    (Önce SEQ küçük, eşitlikte SHARE büyük tercih edilir.)
# ---------------------------------------------------------
sort_cols = ["MU_GLOBAL"]
if "SEQ" in df.columns:   sort_cols.append("SEQ")
if "SHARE" in df.columns: sort_cols.append("SHARE")

dom = (df.sort_values(sort_cols, ascending=[True, True, False])
         .groupby("MU_GLOBAL", as_index=False)
         .first())

# ---------------------------------------------------------
# 7) ÖZETLEYİCİ: insan-dili rapor (pH/OC/Kil/AWC/Drenaj vs.)
# ---------------------------------------------------------
def summarize_soil(row: pd.Series) -> str:
    # pH (üst)
    ph = _num(row.get("T_PH_H2O"))
    if ph is not None:
        if   ph < 5.5: ph_txt = "asidik"
        elif ph < 6.5: ph_txt = "hafif asidik"
        elif ph < 7.5: ph_txt = "nötr–hafif alkali"
        elif ph < 8.5: ph_txt = "alkali"
        else:          ph_txt = "kuvvetli alkali"
    else:
        ph_txt = "bilinmiyor"

    # Organik karbon (üst, %)
    oc = _num(row.get("T_OC"))
    if oc is not None:
        if   oc < 1: oc_txt = "düşük"
        elif oc < 2: oc_txt = "orta"
        else:        oc_txt = "yüksek"
    else:
        oc_txt = "bilinmiyor"

    # Kil (üst, %)
    clay = _num(row.get("T_CLAY"))
    if clay is not None:
        if   clay < 15: clay_txt = "çok kumlu/az kil"
        elif clay < 35: clay_txt = "orta dokulu"
        else:           clay_txt = "killi"
    else:
        clay_txt = "bilinmiyor"

    # AWC (mm/m)
    awc = _num(row.get("AWC_MM_PER_M"))
    if awc is not None:
        if   awc < 100:  awc_txt = f"düşük ({awc} mm/m)"
        elif awc < 150:  awc_txt = f"orta ({awc} mm/m)"
        else:            awc_txt = f"yüksek ({awc} mm/m)"
    else:
        awc_txt = "bilinmiyor"

    # Diğer etiketler
    drn_txt = row.get("DRAINAGE_DESC", "bilinmiyor")
    t_usda  = row.get("T_USDA_TEX_DESC") or row.get("T_TEXTURE_DESC")
    s_usda  = row.get("S_USDA_TEX_DESC")
    fao90   = row.get("FAO90_DESC")

    ece = _num(row.get("T_ECE"))
    esp = _num(row.get("T_ESP"))

    lines = []
    if pd.notna(row.get("MU_GLOBAL")):
        lines.append(f"MU_GLOBAL: {int(row['MU_GLOBAL'])}")
    if fao90:  lines.append(f"FAO-90 toprak grubu: {fao90}")
    if t_usda: lines.append(f"Üst toprak USDA doku: {t_usda}")
    if s_usda: lines.append(f"Alt toprak USDA doku: {s_usda}")
    lines.append(f"pH (üst): {ph} → {ph_txt}")
    lines.append(f"Organik karbon (üst): {oc}% → {oc_txt}")
    lines.append(f"Kil (üst): {clay}% → {clay_txt}")
    lines.append(f"Kullanılabilir su kapasitesi (AWC): {awc_txt}")
    lines.append(f"Drenaj: {drn_txt}")
    if ece is not None: lines.append(f"Elektriksel iletkenlik (üst): {ece} dS/m")
    if esp is not None: lines.append(f"Değişebilir sodyum oranı (üst): {esp}%")
    return "\n".join(lines)

# ---------------------------------------------------------
# 8) KULLANICI ARAYÜZÜ: koordinattan sonuç döndüren fonksiyonlar
# ---------------------------------------------------------
def hwsd_point_row(lat: float, lon: float, columns="all") -> pd.Series:
    """
    (lat,lon) → MU_GLOBAL → 'dom' tablosundan satır döndürür.
    columns:
      - "all"      : tüm kolonlar
      - "summary"  : kısa özet kolonlar
      - list[str]  : özel kolon listesi
    """
    mu = mu_global_from_latlon(lat, lon, RASTER)
    row = dom.loc[dom["MU_GLOBAL"] == mu]
    if row.empty:
        raise LookupError(f"MU_GLOBAL={mu} için kayıt yok.")
    if columns == "all":
        s = row.iloc[0].copy()
    elif columns == "summary":
        wanted = [
            "MU_GLOBAL","SEQ","SHARE",
            "T_TEXTURE","T_TEXTURE_DESC","T_USDA_TEX_CLASS","T_USDA_TEX_DESC",
            "S_USDA_TEX_CLASS","S_USDA_TEX_DESC",
            "AWC_CLASS","AWC_MM_PER_M","DRAINAGE","DRAINAGE_DESC",
            "T_CLAY","T_SILT","T_SAND","T_OC","T_PH_H2O","T_CEC_SOIL",
            "S_CLAY","S_OC","S_PH_H2O","S_CEC_SOIL",
            "SU_CODE90","FAO90_DESC","T_ECE","T_ESP"
        ]
        keep = [c for c in wanted if c in row.columns]
        s = row.iloc[0][keep].copy()
    else:
        keep = [c for c in columns if c in row.columns]
        s = row.iloc[0][keep].copy()
    s["LAT"] = lat
    s["LON"] = lon
    return s

def hwsd_point_report(lat: float, lon: float, columns="all"):
    """
    (lat,lon) için:
      - Metin raporu (okunabilir özet)
      - İstenen kolonlara göre satır (Series) döndürür.
    """
    s = hwsd_point_row(lat, lon, columns=columns)
    txt = summarize_soil(s)
    return txt, s

# ---------------------------------------------------------
# 9) ÖRNEK ÇALIŞTIRMA
# ---------------------------------------------------------
txt, row = hwsd_point_report(36.0, 36.0, columns="all")  # İstersen columns="summary"
print(txt)
display(row.to_frame("value"))


C:\Users\ataka\AppData\Local\Temp\ipykernel_2240\1442108635.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(f"SELECT * FROM {table_name}", cn)


MU_GLOBAL: 3280
Üst toprak USDA doku: clay (light)
Alt toprak USDA doku: clay (light)
pH (üst): 7.9 → alkali
Organik karbon (üst): 0.75% → düşük
Kil (üst): 55.0% → killi
Kullanılabilir su kapasitesi (AWC): orta (125.0 mm/m)
Drenaj: Poor
Elektriksel iletkenlik (üst): 0.2 dS/m
Değişebilir sodyum oranı (üst): 1.0%


,value
MU_GLOBAL,3280
ID,40779
MU_SOURCE1,3280
MU_SOURCE2,NaN
ISSOIL,1
...,...
AWC_MM_PER_M,125.0
DRAINAGE_DESC,Poor
FAO90_DESC,None
LAT,36.0


In [58]:
# =========================================================
# HWSD ham çıktı + ünitesi + "ne anlama gelir" tablosu
# (yorumsuz/faktüel açıklamalar; sınıflama opsiyonel)
# =========================================================

import pandas as pd

# 1) Alan → {unit, meaning, alias(optional)} sözlüğü
#    Buradaki "meaning" teknik, kısa ve yorumsuz açıklamadır.
SCHEMA = {
    # Kimlik / meta
    "MU_GLOBAL":      {"unit": "-",     "meaning": "HWSD harita birimi kimliği (raster piksel değeri)"},
    "ID":             {"unit": "-",     "meaning": "HWSD_DATA kayıt kimliği"},
    "ISSOIL":         {"unit": "-",     "meaning": "1=toprak; 0=toprak-dışı (göl, şehir, buz vb.)"},

    # Doku (üst)
    "T_SAND":         {"unit": "%",     "meaning": "Üst toprak (0–30 cm) kum oranı"},
    "T_SILT":         {"unit": "%",     "meaning": "Üst toprak (0–30 cm) silt oranı"},
    "T_CLAY":         {"unit": "%",     "meaning": "Üst toprak (0–30 cm) kil oranı"},
    "T_TEXTURE":      {"unit": "kod",   "meaning": "Üst toprak kaba/orta/ince doku sınıfı (kod)"},
    "T_TEXTURE_DESC": {"unit": "-",     "meaning": "Üst toprak kaba/orta/ince doku açıklaması"},
    "T_USDA_TEX_CLASS": {"unit": "kod", "meaning": "Üst toprak USDA doku sınıfı (kod)"},
    "T_USDA_TEX_DESC":  {"unit": "-",   "meaning": "Üst toprak USDA doku sınıfı açıklaması"},

    # Doku (alt)
    "S_SAND":         {"unit": "%",     "meaning": "Alt toprak (30–100 cm) kum oranı"},
    "S_SILT":         {"unit": "%",     "meaning": "Alt toprak (30–100 cm) silt oranı"},
    "S_CLAY":         {"unit": "%",     "meaning": "Alt toprak (30–100 cm) kil oranı"},
    "S_USDA_TEX_CLASS": {"unit": "kod", "meaning": "Alt toprak USDA doku sınıfı (kod)"},
    "S_USDA_TEX_DESC":  {"unit": "-",   "meaning": "Alt toprak USDA doku sınıfı açıklaması"},

    # Kimya / tuzluluk / sodiklik (üst)
    "T_OC":           {"unit": "%",     "meaning": "Üst toprak organik karbon"},
    "T_PH_H2O":       {"unit": "-",     "meaning": "Üst toprak pH (H₂O)"},
    "T_CEC_CLAY":     {"unit": "cmol(+)/kg", "meaning": "Üst toprak kil fraksiyonu katyon değişim kapasitesi"},
    "T_CEC_SOIL":     {"unit": "cmol(+)/kg", "meaning": "Üst toprak toplam katyon değişim kapasitesi"},
    "T_BS":           {"unit": "%",     "meaning": "Üst toprak baz doygunluğu"},
    "T_TEB":          {"unit": "cmol(+)/kg", "meaning": "Üst toprak toplam değişebilir bazlar (Ca+Mg+K+Na)"},
    "T_CACO3":        {"unit": "%",     "meaning": "Üst toprak kalsiyum karbonat (kireç)"},
    "T_CASO4":        {"unit": "%",     "meaning": "Üst toprak kalsiyum sülfat (jips)"},
    "T_ECE":          {"unit": "dS/m",  "meaning": "Üst toprak elektriksel iletkenlik (tuzluluk göstergesi)"},
    "T_ESP":          {"unit": "%",     "meaning": "Üst toprak değişebilir sodyum oranı (sodiklik)"},

    # Kimya (alt)
    "S_OC":           {"unit": "%",     "meaning": "Alt toprak organik karbon"},
    "S_PH_H2O":       {"unit": "-",     "meaning": "Alt toprak pH (H₂O)"},
    "S_CEC_CLAY":     {"unit": "cmol(+)/kg", "meaning": "Alt toprak kil fraksiyonu katyon değişim kapasitesi"},
    "S_CEC_SOIL":     {"unit": "cmol(+)/kg", "meaning": "Alt toprak toplam katyon değişim kapasitesi"},
    "S_BS":           {"unit": "%",     "meaning": "Alt toprak baz doygunluğu"},
    "S_TEB":          {"unit": "cmol(+)/kg", "meaning": "Alt toprak toplam değişebilir bazlar"},
    "S_CACO3":        {"unit": "%",     "meaning": "Alt toprak kalsiyum karbonat (kireç)"},
    "S_CASO4":        {"unit": "%",     "meaning": "Alt toprak kalsiyum sülfat (jips)"},
    "S_ECE":          {"unit": "dS/m",  "meaning": "Alt toprak elektriksel iletkenlik"},
    "S_ESP":          {"unit": "%",     "meaning": "Alt toprak değişebilir sodyum oranı"},

    # Fiziksel diğerleri
    "T_REF_BULK_DENSITY": {"unit": "kg/dm³", "meaning": "Üst toprak referans hacim ağırlığı"},
    "T_BULK_DENSITY":     {"unit": "kg/dm³", "meaning": "Üst toprak hacim ağırlığı"},
    "S_REF_BULK_DENSITY": {"unit": "kg/dm³", "meaning": "Alt toprak referans hacim ağırlığı"},
    "S_BULK_DENSITY":     {"unit": "kg/dm³", "meaning": "Alt toprak hacim ağırlığı"},
    "T_GRAVEL":        {"unit": "% (hacim)", "meaning": "Üst toprak çakıl oranı"},
    "S_GRAVEL":        {"unit": "% (hacim)", "meaning": "Alt toprak çakıl oranı"},

    # Su ve drenaj
    "AWC_CLASS":      {"unit": "kod",   "meaning": "Kullanılabilir su kapasitesi sınıfı (sınıf kodu)"},
    "AWC_MM_PER_M":   {"unit": "mm/m",  "meaning": "Kullanılabilir su kapasitesi (lineer; mm su / m toprak)"},
    "DRAINAGE":       {"unit": "kod",   "meaning": "Drenaj sınıfı (kod)"},
    "DRAINAGE_DESC":  {"unit": "-",     "meaning": "Drenaj sınıfı açıklaması"},

    # Fazlar, kökler, derinlik vb.
    "PHASE1":         {"unit": "kod",   "meaning": "Toprak fazı (ör. taşlı, tuzlu, sodik; kod)"},
    "PHASE1_DESC":    {"unit": "-",     "meaning": "Toprak fazı açıklaması"},
    "PHASE2":         {"unit": "kod",   "meaning": "İkinci faz (kod)"},
    "PHASE2_DESC":    {"unit": "-",     "meaning": "İkinci faz açıklaması"},
    "ROOTS":          {"unit": "kod",   "meaning": "Kök gelişimini sınırlayan durum kodu"},
    "REF_DEPTH":      {"unit": "cm",    "meaning": "Referans derinlik"},
    "DRAINAGE_DESC":  {"unit": "-",     "meaning": "Drenaj açıklaması"},
    "ADD_PROP":       {"unit": "kod",   "meaning": "Ek özellik (ör. petric/gelic/vertic; kod)"},
    "IL":             {"unit": "kod",   "meaning": "İmpermeable layer (su geçirmez tabaka) kodu"},
    "SWR":            {"unit": "kod",   "meaning": "Toprak su rejimi kodu"},

    # Semboller / sınıflandırmalar
    "SU_CODE90":      {"unit": "kod",   "meaning": "FAO-90 toprak kodu"},
    "FAO90_DESC":     {"unit": "-",     "meaning": "FAO-90 toprak sınıfı açıklaması"},
    "SU_CODE":        {"unit": "kod",   "meaning": "Toprak sınıfı genel kodu (kaynak tabanlı)"},
    "SU_SYMBOL":      {"unit": "kod",   "meaning": "Toprak birimi sembolü"},

    # Konum bilgisi (çıktıya eklediklerimiz)
    "LAT":            {"unit": "deg",   "meaning": "Enlem (çıktıya ek)"},
    "LON":            {"unit": "deg",   "meaning": "Boylam (çıktıya ek)"},
}

def fact_table_from_row(s: pd.Series, include_classification: bool=False) -> pd.DataFrame:
    """
    Verilen HWSD satırından "alan | değer | birim | ne anlama gelir" tablosu üretir.
    - 'include_classification=True' yaparsan ek bir 'not(esik)' sütunu ekler (hafif yorumludur).
    """
    records = []
    for key, val in s.items():
        meta = SCHEMA.get(key, {"unit": "-", "meaning": "(tanımsız alan — HWSD özgü)"})
        rec = {
            "Alan": key,
            "Değer (ham)": val,
            "Birim": meta["unit"],
            "Ne anlama gelir": meta["meaning"],
        }
        # İsteğe bağlı: çok hafif sınıflama (yorum istemezsen kapalı kalsın)
        if include_classification:
            rec["Not (eşik)"] = _classify_basic(key, val)
        records.append(rec)
    df_out = pd.DataFrame.from_records(records)
    # Alan adına göre sıralama: önce kimlik ve konum, sonra geri kalanı
    priority = ["MU_GLOBAL","ISSOIL","LAT","LON"]
    df_out["__order"] = df_out["Alan"].apply(lambda x: (0 if x in priority else 1, x))
    df_out = df_out.sort_values("__order").drop(columns="__order")
    return df_out

# (Opsiyonel) çok hafif sınıflama; istemezsen kullanma.
def _to_float(x):
    try:
        return float(str(x).replace(",", "."))
    except Exception:
        return None

def _classify_basic(key, val):
    v = _to_float(val)
    if v is None:
        return None
    if key == "T_PH_H2O":
        if   v < 5.5: return "asidik"
        elif v < 6.5: return "hafif asidik"
        elif v < 7.5: return "nötr–hafif alkali"
        elif v < 8.5: return "alkali"
        else:         return "kuvvetli alkali"
    if key == "T_OC":
        if   v < 1:   return "düşük OC"
        elif v < 2:   return "orta OC"
        else:         return "yüksek OC"
    if key == "T_CLAY":
        if   v < 15:  return "az kil"
        elif v < 35:  return "orta kil"
        else:         return "yüksek kil"
    if key == "AWC_MM_PER_M":
        if   v < 100: return "düşük AWC"
        elif v < 150: return "orta AWC"
        else:         return "yüksek AWC"
    if key in ("T_ECE","S_ECE"):
        return f"{v} dS/m"
    return None

# KULLANIM:
# 'row' ya da 'fullrow' senin az önce aldığın tam satır Seri’si.
# Aşağıdaki çağrı "yorumsuz" tabloyu verir:
ft = fact_table_from_row(row, include_classification=False)
display(ft)

# Eğer hafif eşik notlarını da görmek istersen:
# ft2 = fact_table_from_row(row, include_classification=True)
# display(ft2)


,Alan,Değer (ham),Birim,Ne anlama gelir
4,ISSOIL,1,-,"1=toprak; 0=toprak-dışı (göl, şehir, buz vb.)"
63,LAT,36.0,deg,Enlem (çıktıya ek)
64,LON,36.0,deg,Boylam (çıktıya ek)
0,MU_GLOBAL,3280,-,HWSD harita birimi kimliği (raster piksel değeri)
22,ADD_PROP,3.0,kod,Ek özellik (ör. petric/gelic/vertic; kod)
...,...,...,...,...
35,T_TEB,45.2,cmol(+)/kg,Üst toprak toplam değişebilir bazlar (Ca+Mg+K+Na)
13,T_TEXTURE,3.0,kod,Üst toprak kaba/orta/ince doku sınıfı (kod)
57,T_TEXTURE_DESC,Fine,-,Üst toprak kaba/orta/ince doku açıklaması
27,T_USDA_TEX_CLASS,3.0,kod,Üst toprak USDA doku sınıfı (kod)


In [66]:
# ============================
# Sade "okunur rapor" çıktısı
# ============================

def _fmt(x, unit=""):
    if x is None: return "—"
    try:
        v = float(str(x).replace(",", "."))
        return f"{v:g}{(' ' + unit) if unit else ''}"
    except Exception:
        return str(x)

def soil_brief(lat, lon, extra=False):
    """
    Koordinattaki toprak için yalın rapor yazdırır.
    - Tablosuz, sadece anlamlı göstergeler
    - extra=True → biraz daha teknik detay ekler
    """
    # row: tüm kolonları istersen "summary" da kullanabilirsin
    txt, row = hwsd_point_report(lat, lon, columns="all")

    # Temel alanları topla (olanları göster)
    lines = []
    def add(label, key, unit=""):
        if key in row.index and pd.notna(row[key]):
            lines.append(f"{label}: {_fmt(row[key], unit)}")

    print("───────────── TOPRAK ÖZETİ ─────────────")
    # Sınıf adları
    add("FAO-90 sınıfı", "FAO90_DESC")
    add("Üst toprak USDA doku", "T_USDA_TEX_DESC")
    add("Alt toprak USDA doku", "S_USDA_TEX_DESC")
    if "T_TEXTURE_DESC" in row.index and pd.isna(row.get("T_USDA_TEX_DESC")):
        add("Üst toprak doku (coarse/medium/fine)", "T_TEXTURE_DESC")

    # Fiziksel doku yüzdeleri (üst)
    add("Kum (üst)",  "T_SAND", "%")
    add("Silt (üst)", "T_SILT", "%")
    add("Kil  (üst)", "T_CLAY", "%")

    # Kimya (üst)
    add("pH (üst)",   "T_PH_H2O")
    add("Organik karbon (üst)", "T_OC", "%")
    add("CEC (üst, toprak)", "T_CEC_SOIL", "cmol(+)/kg")
    add("Baz doygunluğu (üst)", "T_BS", "%")
    add("Kireç, CaCO₃ (üst)", "T_CACO3", "%")

    # Su ve drenaj
    add("AWC (kullanılabilir su)", "AWC_MM_PER_M", "mm/m")
    add("Drenaj", "DRAINAGE_DESC")

    # Tuzluluk / sodyum (varsa)
    add("EC (üst, tuzluluk)", "T_ECE", "dS/m")
    add("ESP (üst, sodyum)",  "T_ESP", "%")

    # Konum ve MU
    add("MU_GLOBAL", "MU_GLOBAL")
    add("Enlem", "LAT", "°")
    add("Boylam", "LON", "°")

    # Yazdır
    for ln in lines:
        print(ln)

    if extra:
        print("\n— Ek teknik göstergeler —")
        add("CEC (kil, üst)", "T_CEC_CLAY", "cmol(+)/kg")
        add("Hacim ağırlığı (üst)", "T_BULK_DENSITY", "kg/dm³")
        add("Hacim ağırlığı (alt)", "S_BULK_DENSITY", "kg/dm³")
        add("OC (alt)", "S_OC", "%")
        add("pH (alt)", "S_PH_H2O")
        add("CEC (alt, toprak)", "S_CEC_SOIL", "cmol(+)/kg")
        add("EC (alt)", "S_ECE", "dS/m")
        add("ESP (alt)", "S_ESP", "%")
        for ln in lines[len(lines):]:
            print(ln)  # (ekler zaten print edildi)

# KULLANIM: lat, lon gir
soil_brief(38.2, 39.1)          # yalın
# soil_brief(36.2, 36.1, extra=True)   # daha teknik detaylarla


───────────── TOPRAK ÖZETİ ─────────────
Üst toprak USDA doku: loam
Alt toprak USDA doku: loam
Kum (üst): 43 %
Silt (üst): 34 %
Kil  (üst): 23 %
pH (üst): 7.6
Organik karbon (üst): 1.4 %
CEC (üst, toprak): 15 cmol(+)/kg
Baz doygunluğu (üst): 100 %
Kireç, CaCO₃ (üst): 3.9 %
AWC (kullanılabilir su): 0 mm/m
Drenaj: Imperfectly
EC (üst, tuzluluk): 0.1 dS/m
ESP (üst, sodyum): 4 %
MU_GLOBAL: 3093
Enlem: 38.2 °
Boylam: 39.1 °


In [16]:
# ============================================
# HWSD İÇERİK/ŞEMA ENVANTER RAPORU (KONSOL)
# - Access .mdb içindeki tüm tablolar + sütunlar + satır sayıları
# - HWSD_DATA için özet profil (min/max/NaN/benzersiz vs.)
# - Raster (hwsd.bil) metadata: boyut, çözünürlük, bbox, dtype, nodata
# ============================================

from pathlib import Path
import pyodbc, pandas as pd, numpy as np, textwrap
import rasterio

# ---- 0) Yol ayarı (gerekirse düzenle) ----
BASE   = Path(r"C:/Users/ataka/Desktop/MEHMET/VerimGören/notebooks/hwsd_data")
MDB    = BASE / "HWSD.mdb"
RASTER = BASE / "hwsd.bil"


# ---- 1) Yardımcı fonksiyonlar ----
def connect_access(mdb_path: Path):
    mdb_path = Path(mdb_path).resolve()
    if not mdb_path.exists():
        raise FileNotFoundError(f"Bulunamadı: {mdb_path}")
    conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={str(mdb_path).replace('\\','/')};"
    return pyodbc.connect(conn_str)

def list_tables(cn):
    """Access içindeki tüm kullanıcı tablolarını listeler."""
    # ODBC meta: TABLE, VIEW, SYSTEM TABLE döner; biz TABLE filtreleriz
    tables = []
    cur = cn.cursor()
    for row in cur.tables():
        if row.table_type.upper() == "TABLE":
            tables.append(row.table_name)
    # Sistem tablolarını (MSys*) at
    tables = [t for t in tables if not t.upper().startswith("MSYS")]
    return sorted(tables)

# 1) Eski list_columns'u KALDIR ve yerine bunu koy:
def list_columns_safe(cn, table):
    """
    Access metadata'da UnicodeDecodeError olduğunda
    'SELECT * FROM [table] WHERE 1=0' ile sütunları çekerek güvenli döner.
    """
    cur = cn.cursor()
    try:
        # normal yol (hızlı): ODBC metadata
        cols = []
        for row in cur.columns(table=table):
            cols.append((row.column_name, row.type_name, row.nullable))
        if cols:
            return cols
    except Exception:
        pass  # metadata yöntemi patladı → alta düş

    # yedek yol (güvenli): boş sonuçlu SELECT ile şema
    cur.execute(f"SELECT * FROM [{table}] WHERE 1=0")
    cols = []
    # cur.description: (name, type_code, display_size, internal_size, precision, scale, null_ok)
    for d in (cur.description or []):
        name = d[0]
        # type_code Python tipi olabilir; stringe dökelim
        tname = getattr(d[1], "__name__", str(d[1]))
        nullok = (len(d) > 6 and d[6] is True)
        cols.append((name, tname, nullok))
    return cols

# 2) Döngüde list_columns yerine list_columns_safe kullan:
for t in tabs:
    n = count_rows(cn, t)
    cols = list_columns_safe(cn, t)  # <-- burada değişti
    print(f"— Tablo: {t}  | Satır: {n if n is not None else '?'}  | Sütun: {len(cols)}")
    for (name, typ, nullable) in cols:
        nn = "NULL" if nullable else "NOT NULL"
        print(f"   • {name:25s}  ({typ:12s}, {nn})")
    print()


def count_rows(cn, table):
    cur = cn.cursor()
    try:
        cur.execute(f"SELECT COUNT(*) FROM [{table}]")
        return cur.fetchone()[0]
    except Exception:
        return None

def safe_read_sql(cn, query, max_rows=None):
    """Access'ten DataFrame okur; max_rows verilirse TOP ile sınırlar."""
    if max_rows:
        # Access SQL TOP
        q = query.strip().rstrip(";")
        if q.upper().startswith("SELECT "):
            q = "SELECT TOP {} ".format(max_rows) + q[7:]
    else:
        q = query
    return pd.read_sql(q, cn)

def profile_numeric(df, cols):
    out = []
    for c in cols:
        s = pd.to_numeric(df[c], errors="coerce")
        if s.notna().sum() == 0:
            continue
        out.append({
            "col": c,
            "count": int(s.notna().sum()),
            "unique": int(s.nunique(dropna=True)),
            "min": float(np.nanmin(s)),
            "max": float(np.nanmax(s))
        })
    return out

def profile_non_numeric(df, cols, sample_k=5):
    out = []
    for c in cols:
        s = df[c]
        nn = int(s.notna().sum())
        if nn == 0: 
            continue
        uniq = int(s.nunique(dropna=True))
        sample_vals = list(pd.Series(s.dropna().unique()).astype(str).head(sample_k))
        out.append({"col": c, "count": nn, "unique": uniq, "samples": sample_vals})
    return out

def wrap(txt, width=90):
    return textwrap.fill(str(txt), width=width)

# ---- 2) Raporu üret ----
print("=== HWSD VERİ SETİ İÇERİK/ŞEMA RAPORU ===\n")

# 2a) Access içeriği
cn = connect_access(MDB)
tabs = list_tables(cn)
print(f"[Access] Dosya: {MDB}")
print(f"[Access] Tablolar: {len(tabs)} adet\n")

for t in tabs:
    n = count_rows(cn, t)
    cols = list_columns(cn, t)
    print(f"— Tablo: {t}  | Satır: {n if n is not None else '?'}  | Sütun: {len(cols)}")
    # sütunları kısa yazalım
    for (name, typ, nullable) in cols:
        nn = "NULL" if nullable else "NOT NULL"
        print(f"   • {name:25s}  ({typ:12s}, {nn})")
    print()

# 2b) HWSD_DATA hızlı profil (ilk 200k satırdan numune; Access yavaş olabilir)
if "HWSD_DATA" in tabs:
    print("=== HWSD_DATA Hızlı Profil ===")
    # Access bazen COUNT büyük olunca yavaş; örneklem al
    df_sample = safe_read_sql(cn, "SELECT * FROM HWSD_DATA", max_rows=200000)
    # sütun listesi
    cols = list(df_sample.columns)
    print(f"Sütun sayısı: {len(cols)}")
    print(f"Örneklenen satır: {len(df_sample):,}\n")

    # NaN oranları ve kaç dolu
    nn_stats = df_sample.notna().sum().sort_values(ascending=False)
    print("Dolu değer sayısı (en çoktan aza, ilk 25):")
    for k, v in nn_stats.head(25).items():
        print(f"  - {k:25s}: {int(v):,}")
    print()

    # Sayısal/sayısal olmayan ayır
    num_like = []
    nonnum_like = []
    for c in cols:
        # basit tip tahmini: sayıya coercible ise num_like
        s = pd.to_numeric(df_sample[c], errors="coerce")
        if s.notna().sum() > 0:
            num_like.append(c)
        else:
            nonnum_like.append(c)

    # Temel aralıklar (min-max) — seçili kritik sutunlar öncelikli
    priority = ["T_SAND","T_SILT","T_CLAY","S_SAND","S_SILT","S_CLAY",
                "T_OC","S_OC","T_PH_H2O","S_PH_H2O",
                "T_CEC_SOIL","S_CEC_SOIL","T_ECE","S_ECE","T_ESP","S_ESP",
                "T_CACO3","S_CACO3","AWC_CLASS"]
    num_cols = [c for c in priority if c in num_like] + [c for c in num_like if c not in priority]
    rng = profile_numeric(df_sample, num_cols[:40])  # raporu kısa tutmak için 40 kolon
    if rng:
        print("Sayısal alan min–max (ilk 40):")
        for r in rng:
            print(f"  - {r['col']:15s}: min={r['min']}, max={r['max']}, unique≈{r['unique']}")
        print()

    # Kod sütunları için örnek değerler (lookup'lara gidenler)
    code_cols = [c for c in ["T_TEXTURE","AWC_CLASS","DRAINAGE","PHASE1","PHASE2","SU_CODE90","T_USDA_TEX_CLASS","S_USDA_TEX_CLASS"] if c in cols]
    if code_cols:
        print("Kod sütunlarında örnek değerler:")
        prof = profile_non_numeric(df_sample, code_cols, sample_k=8)
        for r in prof:
            print(f"  - {r['col']:18s}: {r['unique']} farklı kod; örnekler → {', '.join(map(str,r['samples']))}")
        print()

# 2c) Raster metadata
print("=== Raster (hwsd.bil) Metadata ===")
rp = Path(RASTER).resolve()
with rasterio.open(rp) as src:
    print(f"Dosya:   {rp}")
    print(f"Boyut:   width={src.width:,}  height={src.height:,}  bands={src.count}")
    print(f"CRS:     {src.crs}")
    print(f"DType:   {src.dtypes[0]}    NoData: {src.nodata}")
    resx, resy = src.res
    print(f"Çözün.:  {resx}° x {resy}°  (≈30 arc-second)")
    print(f"Bounds:  {src.bounds}")
    # Küçük bir örnek pencerede min/max (global min/max için tüm rasterı okumak pahalı)
    window = rasterio.windows.Window(0,0,min(2000,src.width),min(2000,src.height))
    arr = src.read(1, window=window)
    vmin, vmax = int(np.nanmin(arr)), int(np.nanmax(arr))
    print(f"Örnek pencere MU_GLOBAL min/max: {vmin} / {vmax}")

cn.close()
print("\n— RAPOR TAMAMLANDI —")


— Tablo: D_ADD_PROP  | Satır: 4  | Sütun: 2
   • CODE                       (BYTE        , NULL)
   • VALUE                      (VARCHAR     , NULL)

— Tablo: D_AWC  | Satır: 7  | Sütun: 2
   • CODE                       (INTEGER     , NULL)
   • VALUE                      (VARCHAR     , NULL)

— Tablo: D_COVERAGE  | Satır: 5  | Sütun: 2
   • CODE                       (BYTE        , NULL)
   • VALUE                      (VARCHAR     , NULL)

— Tablo: D_DRAINAGE  | Satır: 7  | Sütun: 2
   • CODE                       (SMALLINT    , NULL)
   • VALUE                      (VARCHAR     , NULL)

— Tablo: D_IL  | Satır: 5  | Sütun: 2
   • CODE                       (BYTE        , NULL)
   • VALUE                      (VARCHAR     , NULL)

— Tablo: D_ISSOIL  | Satır: 2  | Sütun: 2
   • CODE                       (BYTE        , NULL)
   • VALUE                      (VARCHAR     , NULL)

— Tablo: D_PHASE  | Satır: 31  | Sütun: 2
   • CODE                       (BYTE        , NULL)
   • VALUE  

C:\Users\ataka\AppData\Local\Temp\ipykernel_14164\3828751532.py:95: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(q, cn)


Sütun sayısı: 57
Örneklenen satır: 48,148

Dolu değer sayısı (en çoktan aza, ilk 25):
  - ID                       : 48,148
  - ISSOIL                   : 48,148
  - SHARE                    : 48,148
  - SEQ                      : 48,148
  - MU_GLOBAL                : 48,148
  - MU_SOURCE1               : 48,147
  - ADD_PROP                 : 47,981
  - T_TEXTURE                : 47,527
  - T_PH_H2O                 : 47,527
  - REF_DEPTH                : 47,527
  - DRAINAGE                 : 47,527
  - T_CEC_SOIL               : 47,527
  - T_BS                     : 47,527
  - T_TEB                    : 47,527
  - T_ESP                    : 47,527
  - T_SILT                   : 47,526
  - T_CACO3                  : 47,526
  - T_OC                     : 47,526
  - T_CLAY                   : 47,526
  - T_SAND                   : 47,526
  - T_ECE                    : 47,524
  - T_CASO4                  : 47,522
  - AWC_CLASS                : 47,518
  - T_BULK_DENSITY           : 47,514
  

In [20]:
# =========================================================
# HWSD — "Konumdaki verileri göster" (tek hücre, bağımsız)
# =========================================================
from pathlib import Path
import pandas as pd
import rasterio, pyodbc

# ---- 0) YOL AYARI: Kendi dizinine göre düzenle ----
BASE   = Path(r"C:/Users/ataka/Desktop/MEHMET/VerimGören/notebooks/hwsd_data")
MDB    = BASE/"HWSD.mdb"
RASTER = BASE/"hwsd.bil"

# ---- 1) Yardımcılar ----
def read_table_access(table_name, mdb_path=MDB) -> pd.DataFrame:
    mdb_path = Path(mdb_path).resolve()
    if not mdb_path.exists():
        raise FileNotFoundError(f"HWSD.mdb bulunamadı: {mdb_path}")
    conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={str(mdb_path).replace('\\','/')};"
    cn = pyodbc.connect(conn_str)
    try:
        return pd.read_sql(f"SELECT * FROM {table_name}", cn)
    finally:
        cn.close()

def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out.columns = [c.strip().upper() for c in df.columns]
    return out

def get_code_desc_cols(df: pd.DataFrame):
    cols = set(df.columns)
    code = "CODE" if "CODE" in cols else None
    desc = "DESCRIPTION" if "DESCRIPTION" in cols else ("VALUE" if "VALUE" in cols else None)
    return code, desc

def _num(x):
    if x is None: return None
    try: return float(str(x).replace(",", "."))
    except: return None

def mu_global_from_latlon(lat: float, lon: float, raster_path=RASTER) -> int:
    rp = Path(raster_path).resolve()
    with rasterio.open(rp) as src:
        r, c = src.index(lon, lat)  # (x=lon, y=lat)
        mu = int(src.read(1)[r, c])
    if mu <= 0:
        raise ValueError(f"Geçersiz/non-soil MU_GLOBAL={mu}.")
    return mu

# ---- 2) Tabloları oku (ana + gerekli lookup) ----
hwsd = normalize_cols(read_table_access("HWSD_DATA"))

def _safe(name):
    try: return normalize_cols(read_table_access(name))
    except: return None

tex   = _safe("D_TEXTURE")
utex  = _safe("D_USDA_TEX_CLASS")
awc   = _safe("D_AWC")
drn   = _safe("D_DRAINAGE")
sym90 = _safe("D_SYMBOL90")

# ---- 3) Lookup LUT'ları hazırla (varsa) ----
tex_lut = None
if isinstance(tex, pd.DataFrame):
    c,d = get_code_desc_cols(tex)
    if c and d:
        tex_lut = tex.rename(columns={c:"T_TEXTURE", d:"T_TEXTURE_DESC"})[["T_TEXTURE","T_TEXTURE_DESC"]]

usda_t_lut = usda_s_lut = None
if isinstance(utex, pd.DataFrame):
    c,d = get_code_desc_cols(utex)
    if c and d:
        usda_t_lut = utex.rename(columns={c:"T_USDA_TEX_CLASS", d:"T_USDA_TEX_DESC"})[["T_USDA_TEX_CLASS","T_USDA_TEX_DESC"]]
        usda_s_lut = utex.rename(columns={c:"S_USDA_TEX_CLASS", d:"S_USDA_TEX_DESC"})[["S_USDA_TEX_CLASS","S_USDA_TEX_DESC"]]

awc_lut = None
if isinstance(awc, pd.DataFrame):
    c,d = get_code_desc_cols(awc)
    if c and d:
        awc_lut = awc.rename(columns={c:"AWC_CLASS", d:"AWC_MM_PER_M"})[["AWC_CLASS","AWC_MM_PER_M"]]

drn_lut = None
if isinstance(drn, pd.DataFrame):
    c,d = get_code_desc_cols(drn)
    if c and d:
        drn_lut = drn.rename(columns={c:"DRAINAGE", d:"DRAINAGE_DESC"})[["DRAINAGE","DRAINAGE_DESC"]]

sym90_lut = None
if isinstance(sym90, pd.DataFrame):
    c,d = get_code_desc_cols(sym90)
    if c and d:
        sym90_lut = sym90.rename(columns={c:"SU_CODE90", d:"FAO90_DESC"})[["SU_CODE90","FAO90_DESC"]]

# ---- 4) JOIN + numerik zorlamalar + dominant bileşen ----
df = hwsd.copy()
if tex_lut  is not None and "T_TEXTURE"       in df: df = df.merge(tex_lut,  on="T_TEXTURE",       how="left")
if usda_t_lut is not None and "T_USDA_TEX_CLASS" in df: df = df.merge(usda_t_lut, on="T_USDA_TEX_CLASS", how="left")
if usda_s_lut is not None and "S_USDA_TEX_CLASS" in df: df = df.merge(usda_s_lut, on="S_USDA_TEX_CLASS", how="left")
if awc_lut  is not None and "AWC_CLASS"       in df: df = df.merge(awc_lut,  on="AWC_CLASS",       how="left")
if drn_lut  is not None and "DRAINAGE"        in df: df = df.merge(drn_lut,  on="DRAINAGE",        how="left")
if sym90_lut is not None and "SU_CODE90"      in df: df = df.merge(sym90_lut, on="SU_CODE90",       how="left")

# numerik alanları float'a çevir (string gelebilir)
num_cols = [
    "AWC_MM_PER_M","T_PH_H2O","S_PH_H2O","T_OC","S_OC",
    "T_CLAY","T_SILT","T_SAND","S_CLAY","S_SILT","S_SAND",
    "T_ECE","S_ECE","T_ESP","S_ESP","T_CEC_SOIL","S_CEC_SOIL",
    "T_CEC_CLAY","S_CEC_CLAY","T_BS","S_BS","T_TEB","S_TEB","T_CACO3","S_CACO3",
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", "."), errors="coerce")

dom = (df.sort_values(["MU_GLOBAL","SEQ","SHARE"], ascending=[True,True,False])
         .groupby("MU_GLOBAL", as_index=False)
         .first())

# ---- 5) SCHEMA (kısa anlamlar ve birimler) ----
SCHEMA = {
    "FAO90_DESC":{"unit":"-","meaning":"FAO-90 toprak sınıfı açıklaması"},
    "T_USDA_TEX_DESC":{"unit":"-","meaning":"Üst toprak USDA doku sınıfı"},
    "S_USDA_TEX_DESC":{"unit":"-","meaning":"Alt toprak USDA doku sınıfı"},
    "T_TEXTURE_DESC":{"unit":"-","meaning":"Üst toprak kaba/orta/ince doku"},
    "T_SAND":{"unit":"%","meaning":"Üst toprak kum oranı"}, "T_SILT":{"unit":"%","meaning":"Üst toprak silt oranı"}, "T_CLAY":{"unit":"%","meaning":"Üst toprak kil oranı"},
    "T_PH_H2O":{"unit":"-","meaning":"Üst toprak pH (H₂O)"}, "T_OC":{"unit":"%","meaning":"Üst toprak organik karbon"},
    "T_CEC_SOIL":{"unit":"cmol(+)/kg","meaning":"Üst toprak katyon değişim kapasitesi"},
    "T_CACO3":{"unit":"%","meaning":"Üst toprak kireç (CaCO₃)"},
    "AWC_MM_PER_M":{"unit":"mm/m","meaning":"Kullanılabilir su kapasitesi"},
    "DRAINAGE_DESC":{"unit":"-","meaning":"Drenaj açıklaması"},
    "T_ECE":{"unit":"dS/m","meaning":"Üst toprak elektriksel iletkenlik"}, "T_ESP":{"unit":"%","meaning":"Üst toprak değişebilir sodyum oranı"},
    "S_OC":{"unit":"%","meaning":"Alt toprak organik karbon"}, "S_PH_H2O":{"unit":"-","meaning":"Alt toprak pH (H₂O)"},
    "S_CEC_SOIL":{"unit":"cmol(+)/kg","meaning":"Alt toprak katyon değişim kapasitesi"},
    "S_SAND":{"unit":"%","meaning":"Alt toprak kum oranı"}, "S_SILT":{"unit":"%","meaning":"Alt toprak silt oranı"}, "S_CLAY":{"unit":"%","meaning":"Alt toprak kil oranı"},
    "MU_GLOBAL":{"unit":"-","meaning":"HWSD harita birimi"},
    "LAT":{"unit":"deg","meaning":"Enlem"}, "LON":{"unit":"deg","meaning":"Boylam"},
}

# ---- 6) Raporlayıcı ve gösterici ----
def hwsd_point_row(lat, lon) -> pd.Series:
    mu = mu_global_from_latlon(lat, lon, RASTER)
    row = dom.loc[dom["MU_GLOBAL"]==mu]
    if row.empty:
        raise LookupError(f"MU_GLOBAL={mu} için kayıt yok.")
    s = row.iloc[0].copy()
    s["LAT"], s["LON"] = lat, lon
    return s

def show_soil_at(lat, lon):
    row = hwsd_point_row(lat, lon)
    # Kısa özet (var olanları sırayla yaz)
    def add_if(key, label=None, unit=None):
        if key in row.index and pd.notna(row[key]):
            lab = label or key
            u = unit if unit is not None else SCHEMA.get(key,{}).get("unit","")
            u = f" {u}" if u and u!="- " and u!="-" else ""
            print(f"{lab}: {row[key]}{u}")

    print("──────── TOPRAK ÖZETİ ────────")
    add_if("FAO90_DESC", "FAO-90 sınıfı")
    add_if("T_USDA_TEX_DESC", "Üst USDA doku")
    add_if("S_USDA_TEX_DESC", "Alt USDA doku")
    if pd.isna(row.get("T_USDA_TEX_DESC")):
        add_if("T_TEXTURE_DESC", "Üst doku (coarse/medium/fine)")

    add_if("T_SAND","Kum (üst)","%"); add_if("T_SILT","Silt (üst)","%"); add_if("T_CLAY","Kil (üst)","%")
    add_if("T_PH_H2O","pH (üst)"); add_if("T_OC","Organik karbon (üst)","%")
    add_if("T_CEC_SOIL","CEC (üst)","cmol(+)/kg"); add_if("T_CACO3","CaCO₃ (üst)","%")
    add_if("AWC_MM_PER_M","AWC","mm/m"); add_if("DRAINAGE_DESC","Drenaj")
    add_if("T_ECE","EC (üst)","dS/m"); add_if("T_ESP","ESP (üst)","%")

    print("— Konum —")
    add_if("MU_GLOBAL","MU_GLOBAL")
    add_if("LAT","Enlem","deg"); add_if("LON","Boylam","deg")

    # İstersen tüm dolu alanlar + anlam (metin)
    print("\n— Dolu alanlar (alan = değer [birim] → anlam) —")
    for key, val in row[row.notna()].items():
        meta = SCHEMA.get(key, {"unit":"", "meaning":"(HWSD alanı; açıklama sözlükte yok)"})
        unit = meta.get("unit","")
        unit_str = f" {unit}" if unit and unit!="-" else ""
        print(f"{key} = {val}{unit_str}  →  {meta.get('meaning','')}")

# ---- 7) KULLANIM: Koordinatını yaz ----
show_soil_at(36.2, 36.1)
# show_soil_at(41.04, 28.99)


C:\Users\ataka\AppData\Local\Temp\ipykernel_14164\778694272.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(f"SELECT * FROM {table_name}", cn)


──────── TOPRAK ÖZETİ ────────
Üst USDA doku: clay (light)
Alt USDA doku: clay (light)
Kum (üst): 16.0 %
Silt (üst): 29.0 %
Kil (üst): 55.0 %
pH (üst): 7.9
Organik karbon (üst): 0.75 %
CEC (üst): 44.0 cmol(+)/kg
CaCO₃ (üst): 2.5 %
AWC: 125.0 mm/m
Drenaj: Poor
EC (üst): 0.2 dS/m
ESP (üst): 1.0 %
— Konum —
MU_GLOBAL: 3280
Enlem: 36.2 deg
Boylam: 36.1 deg

— Dolu alanlar (alan = değer [birim] → anlam) —
MU_GLOBAL = 3280  →  HWSD harita birimi
ID = 40779  →  (HWSD alanı; açıklama sözlükte yok)
MU_SOURCE1 = 3280  →  (HWSD alanı; açıklama sözlükte yok)
ISSOIL = 1  →  (HWSD alanı; açıklama sözlükte yok)
SHARE = 40.0  →  (HWSD alanı; açıklama sözlükte yok)
SEQ = 1  →  (HWSD alanı; açıklama sözlükte yok)
SU_SYM74 = Vc  →  (HWSD alanı; açıklama sözlükte yok)
SU_CODE74 = 29.0  →  (HWSD alanı; açıklama sözlükte yok)
T_TEXTURE = 3.0  →  (HWSD alanı; açıklama sözlükte yok)
DRAINAGE = 2.0  →  (HWSD alanı; açıklama sözlükte yok)
REF_DEPTH = 100.0  →  (HWSD alanı; açıklama sözlükte yok)
AWC_CLASS = 2.0